# Get the Token for Service Administrator or User

This notebook is called by another notebook that needs a token.  It creates a token for for use by a service administraor who creates and manages services or for use by users who call the services.  

### Expected Inputs and Outputs

usertype - Notebook expects this variable to be populate with a string depending on the level of authentication expected.  It should have a value of "admin" or "user".

token - If the token is successfully generated, then a string token will be placed in this variable and available to the calling notebook.  If an error occurs then a value of "TokenNotCreated" is returned.

### General
Each service is associated with a single SQL statement. Authenticated users of web, mobile, or cloud applications can use these REST endpoints from any REST HTTP client without having to install any Db2 drivers.

The Db2 REST server accepts an HTTP request, processes the request body, and returns results in JavaScript Object Notation (JSON).  

This notebook is used as example for the db2Dean article for http://www.db2dean.com/Previous/Db2RESTdbUpdates.html

You can find more information about this service at: https://www.ibm.com/support/producthub/db2/docs/content/SSEPGG_11.5.0/com.ibm.db2.luw.admin.rest.doc/doc/c_rest.html.

## Finding the Db2 RESTful Endpoint Service API Documentation
The APIs used in this notebook are documented in the container for the endpoint.  If you are running a browser on the host containing the container, you can view the documentation using "localhost" host name.  If that is your case then you can view the documentaiton by pasting this  URL into your browser:  https://localhost:50050/docs  Otherwise, you would substitute the remote IP or host name if the container is on another host.  You would also change https to http if you are running the service in http mode.

## Import the required programming libraries
The requests library is the minimum required by Python to construct RESTful service calls. 

In [ ]:
import requests

## Create the Header File required for getting an authetication token
The RESTful call to the Db2 RESTful Endpoint service is contructed and transmitted as JSON. The first part of the JSON structure is the headers that define the content type of the request.

In [ ]:
headers = {
  "content-type": "application/json"
}

## RESTful Host
The next part defines where the request is sent to. It provides the location of the RESTful service for our calls.  In my case I was running this notebook on the same machine as the REST Endpoint container was running.  If you are on a different host you would need to replace "localhost" with the actual host name or IP.  If you created your Endpoint container using https, then you need to previx the URL with that instead of http.

In [ ]:
Db2RESTful = "http://localhost:50050"

## API Authentication Service
Each service has its own path in the RESTful call. For authentication we need to point to the `v1/auth` service.

In [ ]:
API_Auth = "/v1/auth"

## Authentication
To authenticate to the RESTful service you must provide the connection information for the database you want to query along with the database userid and password. You can also provide an expiry time so that the access token that gets returned will be invalidated after that time period.  Note that the dbHost you specify here must be known inside of the Endpoint container.  


### Determine type of user for token creation and generate the body with the needed information

In [ ]:
token="TokenNotCreated"
try: usertype
except NameError: print("You must provide a user type")

# Significant block
# of code to auhenticate user
# before providing a token  

if usertype == "admin":
    dbusername="service_admin1"
    dbpassword="password"
    print("Creating token for the service administrator.")
else:
    dbusername="service_user1"
    dbpassword="password"
    print("Creating token for the service user.")

#

In [ ]:
#  Run this cell to create the body of the service call to careate a token that expires in 1 hour

body = {
  "dbParms": {
    "dbHost": "172.18.50.240",
    "dbName": "SAMPLE",
    "dbPort": 50000,
    "isSSLConnection": False,
    "username": dbusername,
    "password": dbpassword
  },
  "expiryTime": "1h"
}

## API Service
When communicating with the RESTful service, you must provide the name of the service that you want to interact with. In this case the authentication service is */v1/auth*.   When the cell below is run, the server will establish a connection to the database server.

In [ ]:
try:
    response = requests.post("{}{}".format(Db2RESTful,API_Auth), headers=headers, json=body)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))

A response code of 200 means that the authentication worked properly, otherwise the error that was generated is printed.

In [ ]:
print("Status of token request.  200=Success")
print(response)
print(response.status_code)

The response includes a connection token that is reused throughout the rest of this lab. It ensures secure a connection without requiring that you reenter a userid and password with each request.  

In [ ]:
if (response.status_code == 200):
  token = response.json()["token"]
#  print("Token: {}".format(token))
else: 
  print(response.json())

In [ ]:
print(token)

At this point you can copy the above token to the clip board and paste it into the next notebook.